In [1]:
import os 
import glob
from pathlib import Path
import pandas as pd 
import numpy as np 
import ast
import json 

In [2]:
map_dti = pd.read_csv('/working/lupolab/julia/tcia_analysis/code/dti_seg_map.csv', index_col = 0)

In [3]:
map_dti.head()

,cohort,dti,exam,patient_id,segmentation_name
0,TCGA-LGG-nifti,"['400-DTI_2.6mm_WH-19253.nii.gz', '4-DTI_2.6mm...",08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436,TCGA-DU-5853,6-AXIAL_FLAIR-84159
1,TCGA-LGG-nifti,"['205-dadc-52244.nii.gz', '205-dadc-52244a.nii...",06-01-2000-MRI_BRAIN_per_R-97608,TCGA-CS-6186,401-T2_AX_FLAIR-72445
2,TCGA-LGG-nifti,"['12-ep2ddiffmddw20p2_wipADC-35015a.nii.gz', '...",04-23-2004-NR_MRI_BRAIN_WWO-08141,TCGA-FG-A6J1,17-FLAIR_axial-36217
3,TCGA-LGG-nifti,"['901-PROP_eADC-45332.nii.gz', '900-PROP_ADC-5...",08-16-1995-MRI_BRAIN_WWO_CONTRAST-29908,TCGA-HT-7684,7-Ax_Flair_irFSE_H-26694
4,TCGA-LGG-nifti,"['601-DTI_ASSET-84919a.nii.gz', '601-DTI_ASSET...",06-18-1989-MRI_BRAIN_WWO_CONTRAS-16160,TCGA-DU-7015,8-AXIAL_FLAIR-42112


## Here we're going to figure out which came with preprocessed ADC: 

In [4]:
map_dti['adc'] = ['' for x in range(map_dti.shape[0])]

In [5]:
for idx, row in map_dti.iterrows(): 
    list_of_dti = ast.literal_eval(row['dti'])
    adc_list = [x for x in list_of_dti if 'adc' in x.lower() or 'apparent' in x.lower() or 'coeff' in x.lower() or 'average' in x.lower()]
    if adc_list: 
        row['adc'] = adc_list
    else: 
        row['adc'] = float("NaN")

In [6]:
map_dti['adc'].isnull().value_counts()

False    101
True      54
Name: adc, dtype: int64

In [7]:
map_dti.head()

,cohort,dti,exam,patient_id,segmentation_name,adc
0,TCGA-LGG-nifti,"['400-DTI_2.6mm_WH-19253.nii.gz', '4-DTI_2.6mm...",08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436,TCGA-DU-5853,6-AXIAL_FLAIR-84159,[TCGA-DU-5853_adc.nii.gz]
1,TCGA-LGG-nifti,"['205-dadc-52244.nii.gz', '205-dadc-52244a.nii...",06-01-2000-MRI_BRAIN_per_R-97608,TCGA-CS-6186,401-T2_AX_FLAIR-72445,"[205-dadc-52244.nii.gz, 205-dadc-52244a.nii.gz]"
2,TCGA-LGG-nifti,"['12-ep2ddiffmddw20p2_wipADC-35015a.nii.gz', '...",04-23-2004-NR_MRI_BRAIN_WWO-08141,TCGA-FG-A6J1,17-FLAIR_axial-36217,"[12-ep2ddiffmddw20p2_wipADC-35015a.nii.gz, 10-..."
3,TCGA-LGG-nifti,"['901-PROP_eADC-45332.nii.gz', '900-PROP_ADC-5...",08-16-1995-MRI_BRAIN_WWO_CONTRAST-29908,TCGA-HT-7684,7-Ax_Flair_irFSE_H-26694,"[901-PROP_eADC-45332.nii.gz, 900-PROP_ADC-5240..."
4,TCGA-LGG-nifti,"['601-DTI_ASSET-84919a.nii.gz', '601-DTI_ASSET...",06-18-1989-MRI_BRAIN_WWO_CONTRAS-16160,TCGA-DU-7015,8-AXIAL_FLAIR-42112,[TCGA-DU-7015_adc.nii.gz]


## Here are the manually processed DTI: 

In [8]:
for idx, row in map_dti.iterrows(): 
    
    path = '/working/lupolab/julia/tcia_analysis/datasets/'+'-'.join(row['cohort'].split('-')[:-1])+'/'+row['patient_id']+'/'+row['exam']
    series_in_exam = os.listdir(path)
    print(series_in_exam)
    if 'diffusion_b=1000' in series_in_exam : 
        row['adc'] = 'diffusion_b=1000'
    elif 'diffusion_b=1200' in series_in_exam: 
        row['adc'] = 'diffusion_b=1200'
    

FileNotFoundError: [Errno 2] No such file or directory: '/working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG/TCGA-DU-5853/08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436'

In [15]:
map_dti.adc.isnull().value_counts()

False    101
True      54
Name: adc, dtype: int64

In [16]:
map_dti_to_align = map_dti.loc[~map_dti.adc.isnull()]
map_dti_to_align.shape

(101, 6)

In [17]:
map_dti_to_align.head(10)

,cohort,dti,exam,patient_id,segmentation_name,adc
0,TCGA-LGG-nifti,"['400-DTI_2.6mm_WH-19253.nii.gz', '4-DTI_2.6mm...",08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436,TCGA-DU-5853,6-AXIAL_FLAIR-84159,diffusion_b=1000
1,TCGA-LGG-nifti,"['205-dadc-52244.nii.gz', '205-dadc-52244a.nii...",06-01-2000-MRI_BRAIN_per_R-97608,TCGA-CS-6186,401-T2_AX_FLAIR-72445,"[205-dadc-52244.nii.gz, 205-dadc-52244a.nii.gz]"
2,TCGA-LGG-nifti,"['12-ep2ddiffmddw20p2_wipADC-35015a.nii.gz', '...",04-23-2004-NR_MRI_BRAIN_WWO-08141,TCGA-FG-A6J1,17-FLAIR_axial-36217,"[12-ep2ddiffmddw20p2_wipADC-35015a.nii.gz, 10-..."
3,TCGA-LGG-nifti,"['901-PROP_eADC-45332.nii.gz', '900-PROP_ADC-5...",08-16-1995-MRI_BRAIN_WWO_CONTRAST-29908,TCGA-HT-7684,7-Ax_Flair_irFSE_H-26694,"[901-PROP_eADC-45332.nii.gz, 900-PROP_ADC-5240..."
4,TCGA-LGG-nifti,"['601-DTI_ASSET-84919a.nii.gz', '601-DTI_ASSET...",06-18-1989-MRI_BRAIN_WWO_CONTRAS-16160,TCGA-DU-7015,8-AXIAL_FLAIR-42112,diffusion_b=1000
6,TCGA-LGG-nifti,"['501-DTI_2.6mm_WH-21034.nii.gz', '500-DTI_2.6...",01-11-1997-MRI_BRAIN_WWO_CONTRAST-98366,TCGA-DU-8164,4-AXIAL_FLAIR-03012,diffusion_b=1000
7,TCGA-LGG-nifti,['600-Exponential_Apparent_Diffusion_Coefficie...,07-26-1997-MRI_BRAIN_WWO_CONTRAST-17402,TCGA-DU-A5TR,5-AXIAL_FLAIR-79506,[600-Exponential_Apparent_Diffusion_Coefficien...
8,TCGA-LGG-nifti,"['900-PROP_ADC-74032a.nii.gz', '9-Prop_DWI-265...",10-28-1995-MRI_BRAIN_FOR_STEREOTACTIC_WWO_CONT...,TCGA-HT-7604,7-Ax_Flair_irFSE_H-59891,"[900-PROP_ADC-74032a.nii.gz, 900-PROP_ADC-7403..."
9,TCGA-LGG-nifti,['300-Exponential_Apparent_Diffusion_Coefficie...,03-12-1996-MRI_BRAIN_FOR_STEREOTACTIC_WWO_CONT...,TCGA-HT-7690,7-Ax_Flair_irFSE_H-88757,[300-Exponential_Apparent_Diffusion_Coefficien...
12,TCGA-LGG-nifti,"['6-DTI_2.6mm_WH-60904.nii.gz', '602-DTI_2.6mm...",11-19-1996-MRI_BRAIN_WWO_CONTRAST-27038,TCGA-DU-8163,8-AXIAL_FLAIR-03695,diffusion_b=1000


In [119]:
# map_dti_to_align.to_csv('/working/lupolab/julia/tcia_analysis/code/aligned_dti.csv', index = False)

## Register the DTI 

In [9]:
map_dti_to_align = pd.read_csv('/working/lupolab/julia/tcia_analysis/code/aligned_dti.csv')

In [10]:
os.chdir('/working/lupolab/julia/tcia_analysis/code')

In [11]:
usable_patient_gbm = pd.read_csv('usable_patient_gbm.csv', index_col = 0)

In [12]:
usable_patient_lgg = pd.read_csv('usable_patient_lgg.csv', index_col = 0)

In [13]:
usable_patients = pd.concat([usable_patient_gbm, usable_patient_lgg], ignore_index=True)

In [14]:
usable_patients.head(1)

,ground_truth,image,sd,patient_id,exam_id,fixed_vol
0,T1C,/working/lupolab/julia/tcia_analysis/datasets/...,701-BRAIN_WWO_T1_COR_POST-08207,TCGA-14-0865,06-04-1995-MRI_BRAIN_W_WOUT_CONTRAST-31488,/working/lupolab/julia/tcia_analysis/datasets/...


In [15]:
PATH = Path('/working/lupolab/julia/tcia_analysis/datasets/')

In [16]:
usable_patients = pd.read_csv(Path(PATH, 'one_exam_per_patient_with_fixed_vol.csv'))

In [17]:
usable_patients.head(1)

,patient_id,cohort,patient_exam,image,contrast,contrast_label,plane,exam_eligible,fixed_vol
0,TCGA-06-0240,TCGA-GBM,07-02-2005-04728,9-AxT1-thin_for_surgery-27598,1,T1C,ax,1,4-AxFLAIR-thin_for_surgery-47036


Need to find the fixed volume using patient id/ exam/ & then just create command to do this with diffusion: 

`BRAINSFit --fixedVolume $fixed_vol.nii.gz --movingVolume $moving_vol.nii.gz --outputVolume $output_vol.nii.gz --outputTransform $transform_output.tfm  --transformType Rigid`

In [18]:
map_dti_to_align['aligned_dti'] = ['' for x in range(map_dti_to_align.shape[0])]

In [19]:
map_dti_to_align.head()

,cohort,dti,exam,patient_id,segmentation_name,adc,aligned_dti
0,TCGA-LGG-nifti,"['400-DTI_2.6mm_WH-19253.nii.gz', '4-DTI_2.6mm...",08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436,TCGA-DU-5853,11-AX_T1_POST_GD_FLAIR-81300a_n,diffusion_b=1000,
1,TCGA-LGG-nifti,['205-dadc-52244.nii.gz'],06-01-2000-MRI_BRAIN_per_R-97608,TCGA-CS-6186,1001-T1_SE_POST-94212a_n,['205-dadc-52244.nii.gz'],
2,TCGA-LGG-nifti,"['12-ep2ddiffmddw20p2_wipADC-35015.nii.gz', '1...",04-23-2004-NR_MRI_BRAIN_WWO-08141,TCGA-FG-A6J1,20-T1fl2dAx__Gd-43129a_n,"['12-ep2ddiffmddw20p2_wipADC-35015.nii.gz', '1...",
3,TCGA-LGG-nifti,"['601-DTI_ASSET-84919.nii.gz', '600-DTI_ASSET-...",06-18-1989-MRI_BRAIN_WWO_CONTRAS-16160,TCGA-DU-7015,10-AX_T1_POST_GD_FLAIR-19430a_n,diffusion_b=1000,
4,TCGA-LGG-nifti,"['501-DTI_2.6mm_WH-21034.nii.gz', '5-DTI_2.6mm...",01-11-1997-MRI_BRAIN_WWO_CONTRAST-98366,TCGA-DU-8164,10-AX_T1_POST_GD_FLAIR-02819a_n,diffusion_b=1000,


In [20]:
commands = []
for idx, row in map_dti_to_align.iterrows(): 
    patient_id = row['patient_id']
    fixed_vol_df = usable_patients.loc[usable_patients.patient_id == patient_id]['fixed_vol'].reset_index()
    
    fixed_vol = (fixed_vol_df['fixed_vol'][0])
    root_dir = '/working/lupolab/julia/tcia_analysis/datasets/'
    cohort = row['cohort']
    exam = row['exam']
    fixed_vol_with_path = Path(root_dir, cohort, patient_id, exam, fixed_vol+'a.nii.gz')
    if 'diffu' in row['adc']: 
        moving_vol = str(Path(root_dir, cohort, patient_id, exam, patient_id+'_adc.nii.gz'))
        output_vol = str(Path(root_dir, cohort, patient_id, exam, patient_id+'_adca.nii.gz'))
        command_align = 'BRAINSFit --fixedVolume '+str(fixed_vol_with_path)+' --movingVolume '+moving_vol+' --outputVolume '+output_vol+' --transformType Rigid'
        commands.append(command_align)
        row['aligned_dti'] = output_vol
    else: 
#         print(type(row['adc']))
#         list_adc = ast.literal_eval(row['adc'])
        output_vols = []
        for series in row['adc']: 
            moving_vol = str(Path(root_dir, cohort, patient_id, exam, series))
            output_vol = str(Path(root_dir, cohort, patient_id, exam, '.'.join(series.split('.')[:-2])+'a.nii.gz'))
            command_align = command_align = 'BRAINSFit --fixedVolume '+fixed_vol+' --movingVolume '+moving_vol+' --outputVolume '+output_vol+' --transformType Rigid'
            commands.append(command_align)
            output_vols.append(output_vol)
        row['aligned_dti'] = output_vols

In [21]:
commands

['BRAINSFit --fixedVolume /working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-5853/08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436/6-AXIAL_FLAIR-84159a.nii.gz --movingVolume /working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-5853/08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436/TCGA-DU-5853_adc.nii.gz --outputVolume /working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-5853/08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436/TCGA-DU-5853_adca.nii.gz --transformType Rigid',
 'BRAINSFit --fixedVolume 401-T2_AX_FLAIR-72445 --movingVolume /working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-CS-6186/06-01-2000-MRI_BRAIN_per_R-97608/[ --outputVolume /working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-CS-6186/06-01-2000-MRI_BRAIN_per_R-97608/a.nii.gz --transformType Rigid',
 "BRAINSFit --fixedVolume 401-T2_AX_FLAIR-72445 --movingVolume /working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-CS-6186/06-01-2000-MRI_BRAIN_per_R-97608/' --

In [22]:
os.chdir('/working/lupolab/julia/tcia_analysis/code/')

In [23]:
x = open('register_ADCs.txt', 'w')

In [24]:
x.write('{')
x.write('\n')
for command in commands: 
    x.write(command)
    x.write('\n')
x.write('}')
x.close()

In [25]:
!grid_pipeline --scriptonly register_ADCs.txt


In [109]:
# Run qsub register_ADC.txt_1.csh on Atom at Sun 3:33 pm Mar 1, 2020
